In [1]:
#连接ERP的DB2数据库，获取TACAIA2（前台：ACAI25）,TACACM4(ACAC80非产成品),TACACMM(ACACMM),TACACJ5(废钢入库履历)
import ibm_db
import pandas as pd
connStr='DATABASE=BHBXA1;HOSTNAME=192.168.2.28;PORT=50000;PROTOCOL=TCPIP;UID=bhbxa1;PWD=bhbxa1'
con=ibm_db.connect(connStr,'','')


In [2]:
import ibm_db_dbi

In [3]:
conn=ibm_db_dbi.Connection(con)

In [125]:
tacaia2=pd.read_sql('select * from tacaia2',conn,parse_dates={'APP_TRNC_TIME':'%Y%m%d%H%M%S'})

In [126]:
from pandas.api.types import CategoricalDtype
tacaia2['ACCOUNT_TITLE_ITEM'].astype(CategoricalDtype(['33','03','31','01','50','51']))

0        33
1        03
2        31
3        01
4        33
5        03
6        33
7        03
8        33
9        03
10       33
11       03
12       33
13       03
14       33
15       03
16       33
17       03
18       33
19       03
20       33
21       03
22       33
23       03
24       31
25       01
26       33
27       03
28       31
29       01
         ..
13284    33
13285    03
13286    31
13287    01
13288    33
13289    03
13290    31
13291    01
13292    33
13293    03
13294    33
13295    03
13296    33
13297    03
13298    33
13299    03
13300    33
13301    03
13302    31
13303    01
13304    33
13305    03
13306    31
13307    01
13308    03
13309    33
13310    03
13311    33
13312    03
13313    33
Name: ACCOUNT_TITLE_ITEM, Length: 13314, dtype: category
Categories (6, object): [33, 03, 31, 01, 50, 51]

In [127]:
tacaia2.set_index(['MAT_NO','ACCOUNT_TITLE_ITEM','UNIT_CODE','SALE_ORDER_SUB_NO'],inplace=True)

In [132]:
tacaia2.index.names

FrozenList(['MAT_NO', 'ACCOUNT_TITLE_ITEM', 'UNIT_CODE', 'SALE_ORDER_SUB_NO'])

In [135]:
tacaia2.sort_index(level=['MAT_NO', 'ACCOUNT_TITLE_ITEM', 'UNIT_CODE', 'SALE_ORDER_SUB_NO'])

MD2005132501                       1   
B1606030FDAA      50                 LA03      MD2004291301                       1   
B1607078FEAEBA    03                 LA01      MD2005094001                           
                  31                 LA01      MD2005094001                           
                  33                 LA01      MD2005094001                           
B1607078FEAEBAAA  01                 LK01      MD2005094001                           
                  50                 LK01      MD2005094001                           
B1607078FEAEBAAB  01                 LK01      MD2005094001                           
B1607078FEAEBBA   03                 LA01      MD2005094001                           
                  31                 LA01      MD2005094001                           
                  33                 LA01      MD2005094001                           
B1607078FEAEBBAAA 01                 LK01      MD2005094001                           
                  50                 LK01      MD2005094001                           
B1607078FEAEBBAAB 01                 LK01      MD2005094001                           
B1607078FEAEBBB   03                 LA01      MD2005094001                           
                  31                 LA01      MD2005094001                           
                  33                 LA01      MD2005094001                           
B1607078FEAEBBBA  01                 LC02      MD2005094001                           
                  50                 LC02      FD2005290103                           
B1607078FEAEBBBB  01                 LC02      MD2005094001                           
                  50                 LC02      FD2005290103                           
B1607078FEAEBBBC  01                 LC02      MD2005094001                           
B1607078FEAEDDA   03                 LA01      MD2005094001                           
                  31                 LA01      MD2005094001                           
                  33                 LA01      MD2005094001                           
B1607078FEAEDDAAA 01                 LK01      MD2005094001                           
...                                                                             ...   
P2005004BAA       31                           LD2005200102                           
P2005004BAAAA     01                 LK01      LD2005200102                           
P2005004BAAAB     01                 LK01      LD2005200102                           
                  03                 LK01      LD2005200103                           
                  33                 LK01      LD2005200103                           
P2005004BAB       31                           LD2005200102                           
P2005004BABAA     01                 LK01      LD2005200102                           
P2005004BABAB     01                 LK01      LD2005200102                           
                  03                 LK01      LD2005200103                           
                  33                 LK01      LD2005200103                           
P2005004BABBA     01                 LK01      LD2005200102                           
P2005004BABBB     01                 LK01      LD2005200102                           
                  03                 LK01      LD2005200103                           
                  33                 LK01      LD2005200103                           
P2005004BABCA     01                 LK01      LD2005200102                           
P2005004BABCB     01                 LK01      LD2005200102                           
                  03                 LK01      LD2005200103                           
                  33                 LK01      LD2005200103                           
P2005004BABD      01                 LI01      LD2005200102                           
P2005004BB        01                         

In [136]:
tacaia2.head()

REC_CREATOR  \
MAT_NO      ACCOUNT_TITLE_ITEM UNIT_CODE SALE_ORDER_SUB_NO               
B2004080BHA 33                 LF04      MD2005061901           001161   
            03                 LF04      MD2005061901           001161   
            31                 LF04      MD2005061901           001161   
            01                 LF04      MD2005061901           001161   
B2003125AFA 33                 LF03      MD2005061901           001161   

                                                           REC_CREATE_TIME  \
MAT_NO      ACCOUNT_TITLE_ITEM UNIT_CODE SALE_ORDER_SUB_NO                   
B2004080BHA 33                 LF04      MD2005061901       20200602195038   
            03                 LF04      MD2005061901       20200602195038   
            31                 LF04      MD2005061901       20200602195038   
            01                 LF04      MD2005061901       20200602195038   
B2003125AFA 33                 LF03      MD2005061901       20200602195038   

                                                           REC_REVISOR  \
MAT_NO      ACCOUNT_TITLE_ITEM UNIT_CODE SALE_ORDER_SUB_NO               
B2004080BHA 33                 LF04      MD2005061901                    
            03                 LF04      MD2005061901                    
            31                 LF04      MD2005061901                    
            01                 LF04      MD2005061901                    
B2003125AFA 33                 LF03      MD2005061901                    

                                                           REC_REVISE_TIME  \
MAT_NO      ACCOUNT_TITLE_ITEM UNIT_CODE SALE_ORDER_SUB_NO                   
B2004080BHA 33                 LF04      MD2005061901                        
            03                 LF04      MD2005061901                        
            31                 LF04      MD2005061901                        
            01                 LF04      MD2005061901                        
B2003125AFA 33                 LF03      MD2005061901                        

                                                           ARCHIVE_FLAG  \
MAT_NO      ACCOUNT_TITLE_ITEM UNIT_CODE SALE_ORDER_SUB_NO                
B2004080BHA 33                 LF04      MD2005061901                     
            03                 LF04      MD2005061901                     
            31                 LF04      MD2005061901                     
            01                 LF04      MD2005061901                     
B2003125AFA 33                 LF03      MD2005061901                     

                                                           COMPANY_CODE  \
MAT_NO      ACCOUNT_TITLE_ITEM UNIT_CODE SALE_ORDER_SUB_NO                
B2004080BHA 33                 LF04      MD2005061901                BH   
            03                 LF04      MD2005061901                BH   
            31                 LF04      MD2005061901                BH   
            01                 LF04      MD2005061901                BH   
B2003125AFA 33                 LF03      MD2005061901                BH   

                                                           COMPANY_CNAME  \
MAT_NO      ACCOUNT_TITLE_ITEM UNIT_CODE SALE_ORDER_SUB_NO                 
B2004080BHA 33                 LF04      MD2005061901                      
            03                 LF04      MD2005061901                      
            31                 LF04      MD2005061901                      
            01                 LF04      MD2005061901                      
B2003125AFA 33                 LF03      MD2005061901                      

                                                             WHOLE_BACKLOG  \
MAT_NO      ACCOUNT_TITLE_ITEM UNIT_CODE SALE_ORDER_SUB_NO                   
B2004080BHA 33                 LF04      MD2005061901       L0L4L1L2L3LK9A   
            03                 LF04      MD2005061901       L0L4L1L2L3LK9A   
            31                

In [137]:
tacaia2

0.0   
B2004038NCD   03                 LA01      MD2004301620         0.0   
B2004080BDB   33                 LA01      MD2004303801         0.0   
              03                 LA01      MD2004303801         0.0   
B1910162IBBBA 33                 LA01      MD2004303802         0.0   
              03                 LA01      MD2004303802         0.0   
B1910162JBA   33                 LA01      MD2004302801         0.0   

                                                             SURFACE_PLATE_CODE  \
MAT_NO        ACCOUNT_TITLE_ITEM UNIT_CODE SALE_ORDER_SUB_NO                      
B2004080BHA   33                 LF04      MD2005061901                       2   
              03                 LF04      MD2005061901                       2   
              31                 LF04      MD2005061901                       2   
              01                 LF04      MD2005061901                       2   
B2003125AFA   33                 LF03      MD2005061901                       2   
              03                 LF03      MD2005061901                       2   
B2004080ACEDC 33                 LF04      MD2005061901                       2   
              03                 LF04      MD2005061901                       2   
B2004080ACEDB 33                 LF04      MD2005061901                       2   
              03                 LF04      MD2005061901                       2   
B2004085AIA   33                 LF04      MD2005061901                       2   
              03                 LF04      MD2005061901                       2   
B2003122AG    33                 LA01      MD2005061902                       2   
              03                 LA01      MD2005061902                       2   
B1912152BEB   33                 LF04      MD2005061902                       1   
              03                 LF04      MD2005061902                       1   
B2004080ACDA  33                 LA01      MD2005061902                       2   
              03                 LA01      MD2005061902                       2   
B2004085AM    33                 LA01      MD2005061902                       2   
              03                 LA01      MD2005061902                       2   
B1912152BEC   33                 LF04      MD2005061902                       1   
              03                 LF04      MD2005061902                       1   
B2004080BDA   33                 LA01      MD2005061902                       2   
              03                 LA01      MD2005061902                       2   
              31                 LA01      MD2005061902                       2   
              01                 LA01      MD2005061902                       2   
B2004085ALA   33                 LF03      MD2005061902                       2   
              03                 LF03      MD2005061902                       2   
              31                 LF03      MD2005061902                       2   
              01                 LF03      MD2005061902                       2   
...                                                                         ...   
B2004111AGP   33                 LA01      MD2005060403                       2   
              03                 LA01      MD2005060403                       2   
              31                 LA01      MD2005060403                       2   
              01                 LA01      MD2005060403                       2   
B2004111AGL   33                 LA01      MD2005060403                       2   
              03                 LA01      MD2005060403                       2   
              31                 LA01      MD2005060403                       2   
              01                 LA01      MD2005060403                       2   
B2004075AH    33                 LA01      MD2005060403                       2   
              03                 LA01      MD2005060403                       2 

In [103]:
tacaia2.to_csv('1.csv')

In [138]:
tacaia2.to_excel('1.xlsx')

In [51]:
#tacaia2.to_excel('tacaia2-202005.xlsx',sheet_name='tacaia2')
#tacaia2.loc[('B2003125AFA','LF03','33','MD2005061901'),'PRODUCT_CODE'][0][:4] #Pandas多层索引的引用
#[x[0] for x in tacaia2.index] #多层索引中一层所有元素的列表
list('330331015051')

['c', 'a', 'b']

In [7]:
acc_period=tacaia2['ACCOUNT_PERIOD'][1]
print(acc_period)

202005


In [6]:
tacacm4=pd.read_sql('select * from tacacm4 where YEAR=? and MON=?',conn,params=[acc_period[:4],acc_period[4:]])

In [7]:
tacacmm=pd.read_sql('select * from tacacmm where YEAR=? and MON=?',conn,params=[acc_period[:4],acc_period[4:]])

In [8]:
tacacj5=pd.read_sql_query('select * from tacacj5 where YEAR_MON=?',conn,params=[acc_period])

In [9]:
print(tacacj5.head())
print(tacacm4.head())
print(tacacmm.head())

REC_CREATOR REC_CREATE_TIME REC_REVISOR REC_REVISE_TIME ARCHIVE_FLAG  \
0      001161  20200601150908                                            
1      001161  20200601150908                                            
2      001161  20200601150908                                            
3      001161  20200601150908                                            
4      001161  20200601150908                                            

  COMPANY_CODE COMPANY_CNAME YEAR_MON PRODUCT_CODE PRODUCT_CODE_CNAME  \
0           BH       浙江宝鸿不锈钢   202005   9001000000                      
1           BH       浙江宝鸿不锈钢   202005   9001100000                      
2           BH       浙江宝鸿不锈钢   202005   9002000000                      
3           BH       浙江宝鸿不锈钢   202005   9002400000                      
4           BH       浙江宝鸿不锈钢   202005   9002405893                      

   CRRT_YEAR_STD_AMT  
0             3.5398  
1             3.5398  
2             4.2035  
3             4.2035  
4  

In [17]:
#将ACAI25表按1、材料号升序；2、账务代码：按33 03 31 01 50 51 的顺序排列；3、销售合同号按升序排列
#tacaia2[tacaia2['MAT_WT'].rank(method='dense',ascending=False)==3]

,REC_CREATOR,REC_CREATE_TIME,REC_REVISOR,REC_REVISE_TIME,ARCHIVE_FLAG,COMPANY_CODE,COMPANY_CNAME,WHOLE_BACKLOG,WHOLE_BACKLOG_ACT,JOB_CODE,...,ORDER_FLAG,CLOSE_FLAG,PROJECT_FLAG,PROJECT_CON_FLAG,PROJECT_CHO_FLAG,AMOUNT,SURFACE_PLATE_CODE,STD_CODE,UNDERLAY_CODE,SLEEVE_STYLE
KEY_SEQ,,,,,,,,,,,,,,,,,,,,,
190,001161,20200602195038,,,,BH,,,,L008,...,1,1,,,,0.0,,F1,AM001,


In [18]:
#对账务代码自定义排序：先设排序的列表；将账务代码列的数据类型改成‘category’（astype)；用set_category，设置账务代码的顺序；用sort_values使排序生效，同时，也将材料号和销售合同升序排列
acc_ttl_list=['33','03','31','01','50','51']
tacaia2['ACCOUNT_TITLE_ITEM']=tacaia2['ACCOUNT_TITLE_ITEM'].astype('category')
tacaia2['ACCOUNT_TITLE_ITEM'].cat.set_category(acc_ttl_list,inplace=True)
tacaia2.sortvalues(['MAT_NO','ACCOUNT_TITLE_ITEM','SALE_ORDER_SUB_NO'],ascending=True,inplace=True)
tacaia2

REC_CREATOR REC_CREATE_TIME REC_REVISOR REC_REVISE_TIME ARCHIVE_FLAG  \
KEY_SEQ                                                                        
10899        001161  20200602195038                                            
10900        001161  20200602195038                                            
10901        001161  20200602195038                                            
10902        001161  20200602195038                                            
10903        001161  20200602195038                                            

        COMPANY_CODE COMPANY_CNAME   WHOLE_BACKLOG WHOLE_BACKLOG_ACT JOB_CODE  \
KEY_SEQ                                                                         
10899             BH                L0L4L1L2L3LK9A              L0L1     LA10   
10900             BH                L0L4L1L2L3LK9A              L0L1     LA10   
10901             BH                L0L4L1L2L3LK9A              L0L1     LA10   
10902             BH                L0L4L1